In [1]:
import os 
os.chdir('..')

from utils import NestedDefaultdict
from loaders import DLTLoader

d = DLTLoader("logs/dlt/HPA_partnumber not updated.dlt")
msgs = d.get_messages()

In [2]:
dlt = NestedDefaultdict(int, 5)


# with open("logs/dlt/compact_preprocess.txt", "w") as file:
for m in msgs:
    if m.type_string.decode('utf-8') != 'log' \
        or m.subtype_string.decode('utf-8') not in ('error', 'fatal'):
        continue 
    
    dlt[m.ecuid][m.apid][m.ctid][m.seid][m.payload_decoded] += 1 

In [3]:
counts = []
with open("logs/dlt/compact_preprocess.txt", "w") as file:
    for ecu_name, ecu in dlt.items():
        for app_name, app in ecu.items():
            for ctx_name, ctx in app.items():
                for ses_name, session in ctx.items():
                    counts.append(len(session))
                    block_name = f"{ecu_name} {app_name} {ctx_name} {ses_name}"
                    # print(block_name)
                    print(f"START {block_name} {len(session)}", file=file)
                    for msg, count in session.items():
                        print(f"{count}: {msg}", file=file)
                    print(f"END", file=file)

In [6]:
len(counts), sum(counts)

(462, 2392)

In [5]:
2392 + 462*2

3316

In [10]:
import tiktoken

tokenizer = tiktoken.encoding_for_model("gpt-4")

In [27]:
text = ''' 
    END HPA KAW_ SFW 1061110
'''
len(tokenizer.encode(text))

15

In [21]:
from utils.text_splitters import DLTBlockSplitter
text_splitter = DLTBlockSplitter(8000)

with open("logs/dlt/compact_preprocess.txt", "r") as file:
    texts = text_splitter.create_documents([file.read()])

In [23]:
texts, len(texts)

([Document(page_content='START HPA VMSS CTX0 1060987 7\n31066: Cannot get() IMURawAngularVelocity from hal-srs-monitor!, error: -10\n31064: Cannot get() IMUAngularVelocityCompensated from hal-srs-monitor!, error: -10\n30210: Cannot get() IMUAngularVelocityDifferentiated from hal-srs-monitor!, error: -10\n30205: Cannot get() IMURawAcceleration from hal-srs-monitor!, error: -10\n30148: Cannot get() IMUAccelerationDifferentiated from hal-srs-monitor!, error: -10\n30147: Cannot get() IMUMeasurementUnitInformation from hal-srs-monitor!, error: -10\n1727: comm-sum service is NOT available!\nEND\nSTART HPA VMSS SFW 1060987 1\n31: IpcStubTransport(vehiclemotionstate.3.9.vehiclemotionstate_default)::Close, Disconnect pulse failed!\nEND\nSTART HPA ADA1 CTX0 1061057 1\n34547: Hib Zen Interface reading GetCurrentOnePilotMode is Failed, error code: kFailure\nEND\nSTART HPA ADA1 SFW 1061057 2\n1: IpcStubTransport(adasactuationarbitrationstatusinterface.1.5.adas_actuationarbitrationstatus_default)::C